In [8]:
target = 'case 3'

match target:
    case 'case 1':
        print('The fist case!')
    case 'case 2':
        print('The second case!')
    case 'case 3':
        print('The third case!')
    case _:
        print('The default case!')

The third case!


In [6]:

from dataclasses import dataclass

@dataclass
class TestMatchCase(object):
    field1: int
    field2: str
    field3: list[float | int]
    # __matchargs__ = ('field1', 'field2', 'field3') # Generates automatically for dataclasses and named tuples


values = TestMatchCase(5, 'option3', [.1, .2, .8])

match values:
    case TestMatchCase(1|2 as val1, val2, val3):
        print(f'Case1 | {val1=}, {val2=}, {val3=}')
    case TestMatchCase(3|4 as val1, val2, val3):
        print(f'Case2 | {val1=}, {val2=}, {val3=}')
    case TestMatchCase(5|6 as val1, val2, val3) if val2 == 'option2' and sum(val3) <= 1:
        print(f'Case3 | {val1=}, {val2=}, {val3=}')
    case TestMatchCase(5|6 as val1, val2, val3) if val2 == 'option3' and sum(val3) <= 1:
        print(f'Case4 | {val1=}, {val2=}, {val3=}')
    case TestMatchCase(5|6 as val1, val2, val3) if val2 == 'option2' or val2 == 'option3' and sum(val3) >= 1:
        print(f'Case5 | {val1=}, {val2=}, {val3=}')
    case _:
        print('Default case')



Case5 | val1=5, val2='option3', val3=[0.1, 0.2, 0.8]


In [82]:
import os
import re

import mne as mne
import numpy as np
import mne
import pickle
import matplotlib.pyplot as plt


import os
import pickle
from dataclasses import dataclass
from typing import Dict, List

import mne
import numpy as np
from mne.decoding import CSP
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline

from combiners import EpochsCombiner
from utils.storage_management import check_path
from utils.data_management import dict2str
import re
from typing import Optional
from mne import EpochsArray

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 7]


In [ ]:
def left_vs_right(
        epochs: dict[str, dict[str, EpochsArray]],
        session: str,
        t_min: Optional[float] = 0.,
        t_max: Optional[float] = 1.,
        f_min: Optional[float] = 5.,
        f_max: Optional[float] = 10.
) -> EpochsCombiner:
    resp_lock_li_epochs = epochs[session]['LI']
    resp_lock_lm_epochs = epochs[session]['LM']
    resp_lock_ri_epochs = epochs[session]['RI']
    resp_lock_rm_epochs = epochs[session]['RM']

    return EpochsCombiner(
        resp_lock_li_epochs,
        resp_lock_lm_epochs,
        resp_lock_ri_epochs,
        resp_lock_rm_epochs
    ).filter(
        l_freq=f_min, h_freq=f_max, skip_by_annotation='edge'
    ).crop(
        tmin=t_min, tmax=t_max
    ).combine(
        (0, 1),
        (2, 3)
    )


def lm_vs_li(
        epochs: dict[str, dict[str, EpochsArray]],
        session: str,
        t_min: Optional[float] = 0.,
        t_max: Optional[float] = 1.,
        f_min: Optional[float] = 5.,
        f_max: Optional[float] = 10.
) -> EpochsCombiner:
    resp_lock_li_epochs = epochs[session]['LI']
    resp_lock_lm_epochs = epochs[session]['LM']

    return EpochsCombiner(
        resp_lock_li_epochs,
        resp_lock_lm_epochs,
    ).filter(
        l_freq=f_min, h_freq=f_max, skip_by_annotation='edge'
    ).crop(
        tmin=t_min, tmax=t_max
    ).combine(
        0,
        1
    )


def rm_vs_ri(
        epochs: dict[str, dict[str, EpochsArray]],
        session: str,
        t_min: Optional[float] = 0.,
        t_max: Optional[float] = 1.,
        f_min: Optional[float] = 5.,
        f_max: Optional[float] = 10.
) -> EpochsCombiner:
    resp_lock_ri_epochs = epochs[session]['RI']
    resp_lock_rm_epochs = epochs[session]['RM']

    return EpochsCombiner(
        resp_lock_ri_epochs,
        resp_lock_rm_epochs,
    ).filter(
        l_freq=f_min, h_freq=f_max, skip_by_annotation='edge'
    ).crop(
        tmin=t_min, tmax=t_max
    ).combine(
        0,
        1
    )

In [38]:
from utils.machine_learning.confusion import ConfusionEstimator
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

ROOT = '../Source/Subjects'
SUBJECT = 'Ga_Fed_06'
LOCK = 'RespCor'
SESSIONS = ['B1', 'B10']
CASE = 'left_vs_right'
T_MIN = -0.5
T_MAX = 0.5
F_MIN = 5.
F_MAX = 7.76

epochs_path = os.path.join(ROOT, SUBJECT, 'Epochs')
epochs = dict()

for epoch_file in os.listdir(epochs_path):
    session = re.findall(r'(_B\d\d?)', epoch_file)[0][1:]
    if LOCK not in epoch_file or session not in SESSIONS:
        continue
    if session not in epochs:
        epochs.update({session: dict()})
    for case in ['LI', 'LM', 'RI', 'RM']:
        if case in epoch_file:
            if case not in epochs[session]:
                epochs[session].update(
                    {case: mne.read_epochs(os.path.join(epochs_path, epoch_file))})
            else:
                print(f'\nThe case \"{case}\" already in epoch {epoch_file}, skipping due to a conflict\n')

combiners = dict()

for session in SESSIONS:
        combiners.update({
            session: {
                'left_vs_right': left_vs_right,
                'lm_vs_li': lm_vs_li,
                'rm_vs_ri': rm_vs_ri
            }[CASE](epochs, session, T_MIN, T_MAX, F_MIN, F_MAX)
        })

del epochs

Reading ../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LI_B10_epochs.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available


/tmp/ipykernel_43495/3764075417.py:28: RuntimeWarning: This filename (../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LI_B10_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  {case: mne.read_epochs(os.path.join(epochs_path, epoch_file))})


Not setting metadata
Not setting metadata
58 matching events found
No baseline correction applied
0 projection items activated
Reading ../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LI_B1_epochs.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Reading ../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LM_B1_epochs.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available


/tmp/ipykernel_43495/3764075417.py:28: RuntimeWarning: This filename (../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LI_B1_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  {case: mne.read_epochs(os.path.join(epochs_path, epoch_file))})
/tmp/ipykernel_43495/3764075417.py:28: RuntimeWarning: This filename (../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LM_B1_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  {case: mne.read_epochs(os.path.join(epochs_path, epoch_file))})


Not setting metadata
Not setting metadata
58 matching events found
No baseline correction applied
0 projection items activated
Reading ../Source/Subjects/Ga_Fed_06/Epochs/RespCor_LM_B10_epochs.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available


...

/home/user/PycharmProjects/FingerMovementDecoder/combiners.py:94: RuntimeWarning: filter_length (1651) is longer than the signal (1001), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_array.copy().filter(l_freq, h_freq, *args, **kwargs)


Setting up band-pass filter from 5 - 7.8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 7.76 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 8.76 Hz)
- Filter length: 1651 samples (1.651 sec)



/home/user/PycharmProjects/FingerMovementDecoder/combiners.py:94: RuntimeWarning: filter_length (1651) is longer than the signal (1001), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_array.copy().filter(l_freq, h_freq, *args, **kwargs)


Setting up band-pass filter from 5 - 7.8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 7.76 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 8.76 Hz)
- Filter length: 1651 samples (1.651 sec)



/home/user/PycharmProjects/FingerMovementDecoder/combiners.py:94: RuntimeWarning: filter_length (1651) is longer than the signal (1001), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_array.copy().filter(l_freq, h_freq, *args, **kwargs)


In [ ]:
from numpy.linalg import LinAlgError
clf = make_pipeline(CSP(n_components=4, reg='shrinkage', rank='full'),
                        LogisticRegression(penalty='l1', solver='saga'))

confusion = {
    'B1_train': dict(tn=list(), fp=list(), fn=list(), tp=list()),
    'B1_test': dict(tn=list(), fp=list(), fn=list(), tp=list()),
    'B10_alien': dict(tn=list(), fp=list(), fn=list(), tp=list()),
    'B10_train': dict(tn=list(), fp=list(), fn=list(), tp=list()),
    'B10_test': dict(tn=list(), fp=list(), fn=list(), tp=list()),
    'B1_alien': dict(tn=list(), fp=list(), fn=list(), tp=list())
}
n_iters = 100
for _ in range(n_iters):
    x_B1 = combiners['B1'].X
    y_B1 = combiners['B1'].Y
    x_B10 = combiners['B10'].X
    y_B10 = combiners['B10'].Y

    x_train_B1, x_test_B1, y_train_B1, y_test_B1 = train_test_split(x_B1, y_B1, train_size=0.75, shuffle=True)
    x_train_B10, x_test_B10, y_train_B10, y_test_B10 = train_test_split(x_B10, y_B10, train_size=0.75, shuffle=True)
    not_fitted = True
    while not_fitted:
        try:
            clf.fit(x_train_B1, y_train_B1)
            not_fitted = False
        except LinAlgError:
            continue
    y_pred_B1_train = clf.predict(x_train_B1)
    y_pred_B1_test = clf.predict(x_test_B1)
    y_pred_B10_alien = clf.predict(x_test_B10)
    not_fitted = True
    while not_fitted:
        try:
            clf.fit(x_train_B10, y_train_B10)
            not_fitted = False
        except LinAlgError:
            continue
    y_pred_B10_train = clf.predict(x_train_B10)
    y_pred_B1_alien = clf.predict(x_test_B1)
    y_pred_B10_test = clf.predict(x_test_B10)
    for pred, actual, dataset_name in zip(
        [y_pred_B1_train, y_pred_B1_test, y_pred_B10_alien, y_pred_B10_train, y_pred_B10_test, y_pred_B1_alien],
        [y_train_B1, y_test_B1, y_test_B10, y_train_B10, y_test_B10, y_test_B1],
        list(confusion.keys())
    ):
        tn, fp, fn, tp = confusion_matrix(pred, actual, labels=[0,1]).ravel()
        confusion[dataset_name]['tn'].append(tn)
        confusion[dataset_name]['fp'].append(fp)
        confusion[dataset_name]['fn'].append(fn)
        confusion[dataset_name]['tp'].append(tp)


measurers = {
    dataset_name: ConfusionEstimator(
        np.array(confusion[dataset_name]['tp']),
        np.array(confusion[dataset_name]['tn']),
        np.array(confusion[dataset_name]['fp']),
        np.array(confusion[dataset_name]['fn'])
    )
    for dataset_name in confusion.keys()
}

Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info


/home/user/PycharmProjects/FingerMovementDecoder/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info


/home/user/PycharmProjects/FingerMovementDecoder/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from 

...

Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info


/home/user/PycharmProjects/FingerMovementDecoder/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.


/home/user/PycharmProjects/FingerMovementDecoder/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank='full'
    MAG: rank 204 from info
Reducing data rank from 204 -> 204
Estimating covariance using SHRINKAGE
Done.


In [79]:
print(f'Comparison between trials {SESSIONS[0]} and {SESSIONS[1]}\n\n'
      f'Subject: {SUBJECT}\n'
      f'Lock: {LOCK}\n'
      f'Case: {CASE}\n'
      f'Time: {T_MIN} - {T_MAX} sec\n'
      f'Frequency: {F_MIN} - {F_MAX} Hz\n'
      f'Number of iterations: {n_iters}\n'
      f'\n###############################'
      )

for dataset_name in confusion.keys():
    print(f'\n----------------------\n{dataset_name}\n----------------------\n\n'
          f'Accuracy: {measurers[dataset_name].acc :.3f}\n'
          f'Specify: {measurers[dataset_name].spec :.3f}\n'
          f'Sensitivity: {measurers[dataset_name].sens :.3f}\n')

Comparison between trials B1 and B10

Subject: Ga_Fed_06
Lock: RespCor
Case: left_vs_right
Time: -0.5 - 0.5 sec
Frequency: 5.0 - 7.76 Hz
Number of iterations: 100

###############################

----------------------
B1_train
----------------------

Accuracy: 0.946
Specify: 0.941
Sensitivity: 0.950


----------------------
B1_test
----------------------

Accuracy: 0.737
Specify: 0.714
Sensitivity: 0.750


----------------------
B10_alien
----------------------

Accuracy: 0.625
Specify: 0.748
Sensitivity: 0.587


----------------------
B10_train
----------------------

Accuracy: 0.920
Specify: 0.911
Sensitivity: 0.929


----------------------
B10_test
----------------------

Accuracy: 0.778
Specify: 0.777
Sensitivity: 0.779


----------------------
B1_alien
----------------------

Accuracy: 0.630
Specify: 0.546
Sensitivity: 0.724

